# Generowanie hydrogramu metodą SCS-CN

Ten notebook pokazuje jak wygenerować hydrogram odpływu ze zlewni przy użyciu metody SCS Curve Number.

## Teoria

Metoda SCS-CN (Soil Conservation Service - Curve Number) to empiryczna metoda obliczania odpływu powierzchniowego. Kluczowe elementy:

- **CN (Curve Number)** - współczynnik 0-100 zależny od użytkowania terenu i typu gleby
- **S** - maksymalna retencja zlewni [mm]
- **Ia** - straty początkowe (zwykle 0.2 × S)
- **Pe** - opad efektywny (powodujący odpływ)

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

# Import bibliotek
from hydrolog.runoff import HydrographGenerator, AMC
from hydrolog.precipitation import BetaHietogram
from hydrolog.time import ConcentrationTime

## 1. Parametry zlewni

Definiujemy podstawowe parametry zlewni (Dopływ spod góry Babica):
- Powierzchnia: 2.56 km²
- Długość najdłuzszej ściezki spływu: 2.53 km
- Spadek cieku: 19.1%
- CN = 57 (tereny rolnicze, gleby grupy B)

In [ ]:
# Parametry zlewni
area_km2 = 2.56
length_km = 2.53
slope = 0.191  # 19.1%
cn = 57

## 2. Obliczenie czasu koncentracji

Czas koncentracji to czas przepływu wody z najdalszego punktu zlewni do przekroju zamykającego.

In [ ]:
# Oblicz czas koncentracji wzorem NRCS
tc = ConcentrationTime.nrcs(length_km=length_km, slope_m_per_m=slope, cn=cn)
print(f"Czas koncentracji (Kirpich): {tc:.1f} min")

## 3. Utworzenie hietogramu

Hietogram opisuje rozkład czasowy opadu. Używamy rozkładu Beta, który dobrze odwzorowuje naturalne burze.

In [ ]:
# Parametry opadu
total_rainfall_mm = 89.2  # Całkowita suma opadu
duration_min = 60.0 * 24  # Czas trwania opadu
timestep_min = 60.0       # Krok czasowy

# Utwórz hietogram Beta
hietogram = BetaHietogram(alpha=2.0, beta=3.0)
precip = hietogram.generate(
    total_mm=total_rainfall_mm,
    duration_min=duration_min,
    timestep_min=timestep_min
)

print(f"Liczba kroków czasowych: {precip.n_steps}")
print(f"Intensywności [mm]: {[f'{x:.2f}' for x in precip.intensities_mm]}")

## 4. Generowanie hydrogramu

Łączymy wszystkie elementy i generujemy hydrogram odpływu.

In [ ]:
# Utwórz generator hydrogramu
generator = HydrographGenerator(
    area_km2=area_km2,
    cn=cn,
    tc_min=tc
)

# Wygeneruj hydrogram dla normalnych warunków wilgotnościowych (AMC-II)
result = generator.generate(precip, amc=AMC.II)

# Wyniki
print("=" * 50)
print("WYNIKI OBLICZEŃ")
print("=" * 50)
print(f"Przepływ maksymalny: {result.peak_discharge_m3s:.2f} m³/s")
print(f"Czas do szczytu: {result.time_to_peak_min:.0f} min")
print(f"Objętość odpływu: {result.total_volume_m3:,.0f} m³")
print(f"Współczynnik odpływu: {result.runoff_coefficient:.3f}")

## 5. Wpływ warunków wilgotnościowych (AMC)

Warunki wilgotnościowe gleby przed opadem mają znaczący wpływ na odpływ:
- **AMC-I** - warunki suche (5-dniowy opad < 35mm)
- **AMC-II** - warunki normalne (domyślne)
- **AMC-III** - warunki mokre (5-dniowy opad > 53mm)

In [ ]:
# Porównanie warunków AMC
print("Porównanie warunków wilgotnościowych:")
print("-" * 60)
print(f"{'AMC':<10} {'Qmax [m³/s]':>15} {'Wsp. odpływu':>15}")
print("-" * 60)

for amc in [AMC.I, AMC.II, AMC.III]:
    result = generator.generate(precip, amc=amc)
    print(f"{amc.name:<10} {result.peak_discharge_m3s:>15.2f} {result.runoff_coefficient:>15.3f}")

## 6. Wizualizacja hydrogramu

In [ ]:
# Opcjonalna wizualizacja (wymaga matplotlib)
try:
    import matplotlib.pyplot as plt
    
    result = generator.generate(precip, amc=AMC.II)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    
    # Hietogram (opad)
    ax1.bar(precip.times_min, precip.intensities_mm, 
            width=timestep_min*0.8, color='steelblue', alpha=0.7)
    ax1.set_ylabel('Opad [mm]')
    ax1.set_title('Hietogram opadu')
    ax1.invert_yaxis()
    ax1.grid(True, alpha=0.3)
    
    # Hydrogram (odpływ)
    ax2.plot(result.hydrograph.times_min, result.hydrograph.discharge_m3s, 
             'b-', linewidth=2, label='Hydrogram')
    ax2.fill_between(result.hydrograph.times_min, result.hydrograph.discharge_m3s, 
                     alpha=0.3, color='steelblue')
    ax2.axhline(y=result.peak_discharge_m3s, color='r', 
                linestyle='--', label=f'Qmax = {result.peak_discharge_m3s:.2f} m³/s')
    ax2.set_xlabel('Czas [min]')
    ax2.set_ylabel('Przepływ [m³/s]')
    ax2.set_title('Hydrogram odpływu')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Matplotlib nie jest zainstalowany. Zainstaluj: pip install matplotlib")

## Podsumowanie

W tym notebooku pokazano:
1. Jak obliczyć czas koncentracji
2. Jak utworzyć hietogram opadu
3. Jak wygenerować hydrogram odpływu metodą SCS-CN
4. Wpływ warunków wilgotnościowych na odpływ

### Dalsze kroki
- Zobacz notebook `02_hietogramy.ipynb` - różne typy hietogramów
- Zobacz notebook `03_czas_koncentracji.ipynb` - porównanie metod